In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=5bc1e9ff6dd8df10d933807fba645106a85c85d696deb7510bb4f8cad1c64458
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F9 '''
def F9(X):
    f = bn.F9()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 84.17528636543352 ,random_state=0 , l1_ratio=0.98989898989899, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_9_2000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_9_400Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,101):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 100, [5] * 100 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 100000)

n_obs , dim =  30, 100
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,300])
for i in range(G[1].shape[0]):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=916827, Mon Apr 13 21:46:10 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 5.015547970497734e+06 1.0e+00 2.40e+00  2e+00  2e+00 0:00.0
    2     34 5.033163176632614e+06 1.0e+00 2.33e+00  2e+00  2e+00 0:00.1
    3     51 4.938014271512935e+06 1.0e+00 2.27e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5038666.07941904] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5045834.79038807] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   60   1020 4.218223795344084e+06 1.3e+00 1.71e+00  2e+00  2e+00 0:03.2
  100   1700 2.243472316830924e+06 1.3e+00 1.01e+00  1e+00  1e+00 0:04.9
  200   3400 -1.240767282033621e+06 1.3e+00 1.43e-01  1e-01  1e-01 0:08.7
  300   5100 -1.330172192594774e+06 1.3e+00 2.56e-02  2e-02  3e-02 0:12.5
  400   6800 -1.333104879588644e+06 1.4e+00 4.53e-03  4e-03  4e-03 0:16.2
  500   8500 -1.333188708907249e+06 1.4e+00 6.95e-04  7e-04  7e-04 0:19.9
  600  10200 -1.333190539976557e+06 1.4e+00 1.19e-04  1e-04  1e-04 0:23.7
  700  11900 -1.333190595592932e+06 1.4e+00 2.07e-05  2e-05  2e-05 0:27.5
  800  13600 -1.333190597579046e+06 1.4e+00 3.61e-06  3e-06  3e-06 0:31.2
  900  15300 -1.333190597642255e+06 1.4e+00 6.53e-07  6e-07  6e-07 0:35.2
 1000  17000 -1.333190597643980e+06 1.4e+00 1.08e-07  1e-07  1e-07 0:39.1
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -1.333190597644026e+06 1.4e+00 2.57e-08  2e-08  2e-08 0:43.0
 1200  20400 -1.333190597644028e+06 1.4e+0

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [4971084.54177121] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=4971084.541771, sigma=2.40e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=4971084.541771, sigma=2.40e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=

    2     34 4.911927950586958e+06 1.0e+00 2.34e+00  2e+00  2e+00 0:00.1
    3     51 4.872160034440421e+06 1.0e+00 2.28e+00  2e+00  2e+00 0:00.1
   75   1275 4.419814790520417e+06 1.2e+00 9.64e-01  9e-01  1e+00 0:03.1
  100   1700 4.341949409292996e+06 1.3e+00 6.71e-01  7e-01  7e-01 0:04.1
  200   3400 4.250871044308151e+06 1.3e+00 2.01e-01  2e-01  2e-01 0:08.3
  300   5100 4.228663582546589e+06 1.7e+00 1.25e-01  1e-01  1e-01 0:12.4
  400   6800 4.224373863315115e+06 1.9e+00 4.44e-02  4e-02  5e-02 0:16.6
  500   8500 4.223761710859152e+06 2.0e+00 1.58e-02  1e-02  2e-02 0:20.8
  600  10200 4.223706443993779e+06 2.0e+00 4.66e-03  4e-03  5e-03 0:24.9
  700  11900 4.223701787366650e+06 2.0e+00 1.25e-03  1e-03  1e-03 0:29.0
  800  13600 4.223701425103088e+06 2.0e+00 3.61e-04  3e-04  4e-04 0:33.2
  900  15300 4.223701400937507e+06 2.1e+00 9.18e-05  8e-05  1e-04 0:37.3
 1000  17000 4.223701397909868e+06 2.3e+00 4.59e-05  4e-05  6e-05 0:41.5
Iterat #Fevals   function value  axis ratio  sigma 

...

 1600  27200 4.223701396909341e+06 3.3e+00 3.64e-08  3e-08  8e-08 1:06.8


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=4223701.396909, sigma=3.56e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1649)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=4223701.396909, sigma=3.42e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1662)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=4223701.396909, sigma=3.41e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1664)
  ')')
/usr/local/li

 1700  28900 4.223701396909341e+06 3.3e+00 2.82e-08  2e-08  6e-08 1:11.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=4223701.396909, sigma=2.83e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1698)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=4223701.396909, sigma=2.83e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1699)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=4223701.396909, sigma=2.84e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1703)
  ')')
/usr/local/li

 1748  29716 4.223701396909341e+06 3.3e+00 2.76e-08  2e-08  6e-08 1:13.0
termination on tolfunhist=1e-12 (Tue Apr 14 00:33:08 2020)
final/bestever f-value = 4.223701e+06 4.223701e+06
incumbent solution: [ 1.31660299 -2.77279213 -2.0018175  -0.6465798   0.66909385 -1.50925515
 -1.07280568  0.09925448 ...]
std deviations: [2.38365904e-08 2.52952681e-08 2.41013970e-08 2.55876687e-08
 2.38278224e-08 2.39197352e-08 2.36002792e-08 2.42395952e-08 ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F9_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F9(X_Values.iloc[i,:100])
    SVM_Fun [i] = F9(X_Values.iloc[i,100:200])
    ELN_Fun [i] = F9(X_Values.iloc[i,200:300])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([8494.13517843]), count=array([1])) 0.00019969011112699786
SVM
ModeResult(mode=array([4021299.84550211]), count=array([1])) 1366919.1670648614
ELN
ModeResult(mode=array([1092682.33701043]), count=array([1])) 157945.65310259556
